In [1]:
import time
import requests
import pandas as pd
import json
from tqdm import tqdm
from datetime import datetime
from multiprocessing.pool import ThreadPool
from requests.adapters import HTTPAdapter
from requests.packages.urllib3.util.retry import Retry

In [2]:
session = requests.Session()
retry = Retry(connect=1000, backoff_factor=0.5)
adapter = HTTPAdapter(max_retries=retry)
session.mount('http://', adapter)
session.mount('https://', adapter)

In [3]:
def start(ticker):
    url = "https://api.upbit.com/v1/candles/minutes/1"
    now = datetime.now().strftime("%Y-%m-%d")
    date_ranges = pd.date_range('2021-01-01', now, freq='3H')
    data_df = pd.DataFrame()
    for date in tqdm(date_ranges):
        params = {
            "market": ticker,
            "count": "200",
            "to": date,
        }
        response = session.get(url, params=params)
        response_dict = json.loads(response.text)
        data_df = pd.concat([data_df, pd.DataFrame(response_dict)])
        time.sleep(0.1)
    data_df = data_df.drop_duplicates(subset=['candle_date_time_kst'])
    datas = data_df.to_dict('records')


    def insert(data):
        insert_url = 'http://127.0.0.1:8000/upbit/candle/'
        session.post(insert_url, data=data)


    pool = ThreadPool(processes=10)
    pool.map(insert, tqdm(datas))
    pool.close()
    pool.join()

In [4]:
ticker_url = 'http://127.0.0.1:8000/upbit/ticker/'

In [5]:
response = session.get('http://127.0.0.1:8000/upbit/ticker/')

In [6]:
ticker_df = pd.DataFrame(json.loads(response.text))

In [7]:
ticker_df

,market,korean_name,english_name
0,BTC-1INCH,1인치네트워크,1inch Network
1,BTC-AAVE,에이브,Aave
2,BTC-ADA,에이다,Ada
3,BTC-AERGO,아르고,Aergo
4,BTC-AHT,아하토큰,AhaToken
...,...,...,...
258,USDT-SC,시아코인,Siacoin
259,USDT-TRX,트론,TRON
260,USDT-TUSD,트루USD,TrueUSD
261,USDT-XRP,리플,Ripple


In [ ]:
for ticker in ticker_df['market']:
    if ticker >= "BTC-ALGO":
        print(ticker)
        start(ticker)